In [ ]:
!pip install lxml

In [ ]:
#Import the required dependencies
import pandas as pd
from urllib.request import Request, urlopen

In [ ]:
from bs4 import BeautifulSoup

# Read the HTML file into a Pandas dataframe
#soup = BeautifulSoup(html_content, 'html.parser')

In [ ]:
from bs4.element import Tag

blocks = ["p", "h1", "h2", "h3", "h4", "h5", "blockquote"]

def _extract_blocks(parent_tag) -> list:
    extracted_blocks = []
    for tag in parent_tag:
        if tag.name in blocks:
            extracted_blocks.append(tag)
            continue
        if isinstance(tag, Tag):
            if len(tag.contents) > 0:
                inner_blocks = _extract_blocks(tag)
                if len(inner_blocks) > 0:
                    extracted_blocks.extend(inner_blocks)
    return extracted_blocks

def to_plaintext(html_text: str) -> str:
    soup = BeautifulSoup(html_text, features="lxml")
    extracted_blocks = _extract_blocks(soup.body)
    extracted_blocks_texts = [block.get_text().strip() for block in extracted_blocks]
    return extracted_blocks_texts



In [ ]:
chittorgarh_data = pd.read_csv('chittorgarh_mainline_30Mar2024_structured_with_links_details.csv')
chittorgarh_data.columns

Index(['mapping_key', 'url', 'Issuer Company', 'Open Date', 'Close Date',
       'Listing Date', 'Issue Price (Rs)', 'Issue Size (Rs Cr.)', 'Lot Size',
       'Exchange', 'subscription_link', 'subscription_info', 'price_info',
       'Date', 'QIB', 'NII', 'Retail', 'Total', 'EMP',
       'bNII (bids above ₹10L)', 'sNII (bids below ₹10L)', 'RII', 'Others',
       'Other', 'Policyholder', 'NSE_symbol', 'BSE_Final_Issue_Price',
       'NSE_Final_Issue_Price', 'BSE_Open', 'NSE_Open', 'BSE_Low', 'NSE_Low',
       'BSE_High', 'NSE_High', 'BSE_Last_Trade', 'NSE_Last_Trade',
       'Total_subscriptions', 'elasticity', 'lot_size_headers',
       'lot_size_rows'],
      dtype='object')

In [ ]:
text_contents_all = []
for url in chittorgarh_data['url'].to_list():
  #Create an abstraction of URL request
  request = Request(url)

  #Add headers to the request
  request.add_header('user-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')

  #Open the URL
  page = urlopen(request)

  #Read HTML content
  html_content = page.read()
  text_blocks = to_plaintext(html_content)
  text_contents_all.append(text_blocks)

In [ ]:
common_texts = set(text_contents_all[0]).intersection(set(text_contents_all[1]))
common_texts

{'',
 '30 days brokerage free trading\nFree - Personal Trading Advisor',
 'Check IPO Performance…',
 'Check Messages by Ranked Members',
 'Company Financials',
 'Enquire Now',
 'FREE Account Opening\nFlat ₹20 Per Trade',
 'FREE Equity Delivery and MF\nFlat ₹20/trade Intra-day/F&O',
 'FREE Intraday Trading (Eq, F&O)Flat ₹20 Per Trade in F&O',
 'Flat ₹20 Per Trade',
 'Free Eq Delivery & MF\nFlat ₹20 Per Trade in F&O',
 'Free Eq Delivery Trades\nFlat ₹20 Per Trade in F&O',
 'Free Equity Delivery\nFlat ₹20 Per Trade in F&O',
 'Lead Manager Reports',
 'Listing Day Trading Information',
 'Open Account',
 'Open FREE Account',
 'Open FREE Demat Account',
 'Open Instant Account',
 'Open Instant Account Now!',
 'Open Online Demat Account',
 'Pay ₹0 brokerage for first 10 days',
 'Pre-Open Session - NSE\nPre-Open Session - BSE',
 'Ranked Members | Members with Most Likes',
 'Read All Reviews\nPost Your Review Manage Reviews',
 'Unlimited @ ₹899/month\nRs 0 Demat AMC',
 'Useful Articles'}

In [ ]:
chittorgarh_data['full_text_content'] = [[j for j in i if j not in common_texts] for i in text_contents_all]

In [ ]:
chittorgarh_data.to_excel('chittorgarh_mainline_30Mar2024_structured_with_links_details_full_text_content.xlsx', index = False)

In [ ]:
ipopremium = pd.read_json('ipopremium.json')
ipopremium

,text,url,stock,spanned,time
0,"[Jun 13, 2022\n - J...",https://ipopremium.in/view/ipo/7,Scarnose International Limited,[],"Jun 13, 2022\n - Ju..."
1,"[May 20, 2022\n - M...",https://ipopremium.in/view/ipo/2,eMUDHRA Limited,[],"May 20, 2022\n - Ma..."
2,"[May 30, 2022\n - J...",https://ipopremium.in/view/ipo/5,Fidel Softech Limited,[],"May 30, 2022\n - Ju..."
3,"[Jun 6, 2022\n - Ju...",https://ipopremium.in/view/ipo/6,Silver Pearl Hospitality & Luxury Spaces Ltd,[],"Jun 6, 2022\n - Jun..."
4,"[May 27, 2022\n - M...",https://ipopremium.in/view/ipo/1,Aether Industries Limited,[],"May 27, 2022\n - Ma..."
...,...,...,...,...,...
389,"[Feb 19, 2024\n - F...",https://ipopremium.in/view/ipo/403,Zenith Drugs (NSE SME),"[KPI, 2023, 2022, 2021, ROE, 35.13%, 29.83%, 4...","Feb 19, 2024\n - Fe..."
390,"[Feb 16, 2024\n - F...",https://ipopremium.in/view/ipo/402,ESCONET TECHNOLOGIES (NSE SME),[Esconet Technologies Limited Financial Inform...,"Feb 16, 2024\n - Fe..."
391,"[Feb 20, 2024\n - F...",https://ipopremium.in/view/ipo/406,Deem Roll Tech (NSE SME),"[Pre IPO, Post IPO, EPS (Rs) , 11.4, 8.92, P/E...","Feb 20, 2024\n - Fe..."
392,"[Feb 21, 2024\n - F...",https://ipopremium.in/view/ipo/404,Juniper Hotels Limited (MAINBOARD),[],"Feb 21, 2024\n - Fe..."


In [ ]:
chittorgarh_data